In [1]:
import pandas as pd

In [5]:
movies = pd.read_csv('../data/ml-1m/movies.dat', sep='::',
                                 names=['MovieID', 'Title', 'Genres'],
                                 engine='python')

In [7]:
movies['MovieID'].max(), movies['MovieID'].min(), movies['MovieID'].shape

(3952, 1, (3883,))

In [8]:
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::',
                                  names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
                                  engine='python')

In [9]:
ratings['MovieID'].max(), ratings['MovieID'].min(), ratings['MovieID'].shape

(3952, 1, (1000209,))